In [2]:
%load_ext autoreload
%autoreload 2
%cd '/home/xingyu/Projects/PlasticineLab'
%pwd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from plb.envs import make
from imitation.train_full import get_args
from plb.engine.taichi_env import TaichiEnv
from plb.optimizer.solver import Solver
from plb.algorithms.logger import Logger


device = 'cuda'

log_dir = './data/connect'
args = get_args("")

obs_channel = len(args.img_mode)
img_obs_shape = (args.image_dim, args.image_dim, obs_channel)

args.env_name ='LiftSpread-v1'
env = make(args.env_name)
env.seed(args.seed)
taichi_env: TaichiEnv = env.unwrapped.taichi_env
T = env._max_episode_steps

/home/xingyu/Projects/PlasticineLab
[Taichi] mode=release
[Taichi] preparing sandbox at /tmp/taichi-d72u8q7_
[Taichi] version 0.7.26, llvm 10.0.0, commit e37bdb5e, linux, python 3.8.5
[Taichi] Starting on arch=cuda
[Tina] version 0.1.1
[Tina] Taichi properties hacked
pimirives: num primitive: 3
Building primitive
action:
  dim: 6
  scale: (0.7, 0.005, 0.005, 0.005, 0.0, 0.0)
collision_group: [0.0, 0.0, 0.0]
color: (0.7568, 0.6039, 0.4196)
friction: 0.9
h: 0.3
init_pos: (0.3, 0.25, 0.5)
init_rot: (0.707, 0.707, 0.0, 0.0)
lower_bound: (0.0, 0.16, 0.0)
r: 0.03
shape: RollingPinExt
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 6
  scale: (0.01, 0.01, 0.0, 0.0, 0.0, 0.05)
collision_group: [0, 0, 1]
color: (0.7568, 0.6039, 0.4196)
friction: 50.0
init_pos: (0.65, 0.02, 0.5)
init_rot: (0.707, 0.707, 0.0, 0.0)
lower_bound: (0.0, 0.0, 0.0)
shape: Box
size: (0.1, 0.1, 0.02)
upper_bound: (1.0, 1.0, 1.0)
variations: None
Building primitive
action:
  dim: 0
  scale:

In [2]:
import matplotlib.pyplot as plt
from imitation.sampler import sample_traj
from imitation.utils import to_action_mask
import numpy as np
from plb.utils.visualization_utils import save_numpy_as_gif
r_key1 = {'init_v': 106, 'target_v': 36, 'contact_loss_mask': [0., 0., 0.]}
r_key2 = {'init_v': 36, 'target_v': 117, 'contact_loss_mask': [ .]}

args.lr = 0.02 # Larger lr (0.1) seems to give pretty good performance (in 30 iterations for 0.1)
args.gd_max_iter = 200
args.num_env = 1 

trajs = []
for i, r_key in enumerate([r_key1, r_key2]):
    filename = 'data/env_modeling/solve_lift_demo_{}.gif'.format(i)
    solver = Solver(args, taichi_env, (0,), return_dist=True)
    if i ==0:
        traj = sample_traj(env, solver, r_key, action_mask=to_action_mask([0, 1]))
    else:
        traj = sample_traj(env, solver, r_key, action_mask=to_action_mask([1, 0]))
    trajs.append(traj)
    save_numpy_as_gif(np.array(traj['obses']), filename)

    from IPython.display import Image
    Image(filename)

{'init_pos': (0.65, 0.08, 0.5), 'radius': 0.05, 'color': 100}
False
Env reseting to:  /home/xingyu/Projects/PlasticineLab/plb/envs/../../datasets/0914_LiftSpread/target/target_36.npy
emd after reset: 0.30352258682250977


199:  6.748153209686279: 100%|██████████| 200/200 [19:04<00:00,  5.72s/it] 


MoviePy - Building file data/env_modeling/solve_lift_demo_0.gif with imageio.


{'init_pos': (0.65, 0.08, 0.5), 'radius': 0.05, 'color': 100}
False
Env reseting to:  /home/xingyu/Projects/PlasticineLab/plb/envs/../../datasets/0914_LiftSpread/target/target_117.npy
emd after reset: 0.42393726110458374


199:  19.070505142211914: 100%|██████████| 200/200 [17:33<00:00,  5.27s/it]


MoviePy - Building file data/env_modeling/solve_lift_demo_1.gif with imageio.


NameError: name 'solver' is not defined

In [2]:
# Generate a lift video
from plb.utils.visualization_utils import save_numpy_as_gif
import numpy as np
filename = 'data/env_modeling/lift_combined.gif'
env.reset(init_v=106, target_v=36)
reset_states = env.get_primitive_state()
a1, a2 = trajs[0]['actions'], trajs[1]['actions']
img_size= 256

frames = []
for i in range(len(a1)):
    env.step(a1[i])
    frames.append(env.taichi_env.render(mode='rgb', img_size=img_size)[:,:,:3])
actions, obses, _ = env.primitive_reset_to(1, reset_states, img_size=img_size)
for frame in obses:
    frames.append(frame[:, :, :3])
for i in range(len(a2)):
    action = np.array([0]* 6 + [-0.7, 0.8, 0., 0., 0., 0.7])
    env.step(a2[i])
    frames.append(env.taichi_env.render(mode='rgb', img_size=img_size)[:,:,:3])
# actions, obses, _ = env.primitive_reset_to(0, reset_states, img_size=img_size)
# for frame in obses:
#     frames.append(frame[:, :, :3])
save_numpy_as_gif(np.array(frames), filename)
demo_actions = np.array(actions)
state = env.get_state()
from IPython.display import Image
Image(filename)

{'init_pos': (0.65, 0.08, 0.5), 'radius': 0.05, 'color': 100}
Env reseting to:  /home/xingyu/Projects/PlasticineLab/plb/envs/../../datasets/0914_LiftSpread/target/target_36.npy
emd after reset: 0.30352258682250977
======================WARNING: contact loss mask not set================


NameError: name 'trajs' is not defined

In [4]:
# Debug actions from replay buffer
dataset_path = 'data/local/0915_LiftSpread_single/0915_LiftSpread_single_2021_09_15_21_14_53_0001/dataset.gz'
from imitation.imitation_buffer import ImitationReplayBuffer
buffer = ImitationReplayBuffer(args)
buffer.load(dataset_path)

Loading dataset in data/local/0915_LiftSpread_single/0915_LiftSpread_single_2021_09_15_21_14_53_0001/dataset.gz
Loading dataset from data/local/0915_LiftSpread_single/0915_LiftSpread_single_2021_09_15_21_14_53_0001/dataset.gz
(50, 12)


In [1]:
# Generate a lift video
from plb.utils.visualization_utils import save_numpy_as_gif
import numpy as np
filename = 'debug.gif'
env.reset(init_v=106, target_v=36)
reset_states = env.get_primitive_state()
img_size= 256
frames = []
actions = buffer.buffer['actions'][:50]
infos = []
for s, action in enumerate(actions):
    obs, r, _, info = env.step(action)
    infos.append(info)
    print('step {}, reward: {}, emd: {}, contact_loss: {}'.format(s, r, info['info_emd'], info['info_contact_loss']))
    frames.append(env.taichi_env.render(mode='rgb', img_size=img_size)[:,:,:3])
emds = np.array([info['info_emd'] for info in infos])
print('emds:', emds)
save_numpy_as_gif(np.array(frames), filename)
demo_actions = np.array(actions)
state = env.get_state()
from IPython.display import Image
Image(filename)

NameError: name 'env' is not defined